In [1]:
import cv2
import pytesseract
from typing import OrderedDict
from math import ceil
import re
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (30, 20)
plt.rcParams['image.cmap'] = 'gray'

In [10]:
data_dict = OrderedDict({
    'Customer':{'value':'','label': 'Customer','Pattern':'[\d]{8}','zoning_type':'in_line'},
    'Invoice':{'value':'','label': 'Invoice','Pattern':'\d{10}', 'zoning_type':'in_line'},
    'Date':{'value':'','label': 'Date','Pattern':'[\d]{2}[\/][\d]{2}[\/][\d]{4}','zoning_type':'in_line'},
    'Accounting CLERK NUMBER':{'value':'','label': 'Accounting','Pattern':'[\d]{3}[-]+[\d]{4}','zoning_type':'in_line'},
    'products':{'value':'','label': 'Products:','Pattern':'[\d]{3}.[0]+','zoning_type':'in_line'},

})


In [4]:
def ocr_the_image(image):
    image = cv2.imread(image)
    image_data = pytesseract.image_to_data(image , output_type=pytesseract.Output.DICT)
    return image_data
extracted_data=ocr_the_image('/home/kavinsharaj/Downloads/NLP HANDS ON (2)/NLP HANDS ON/ADI_d742b4d9-7586-4147-8ae1-b6a6fc0f8e20_1820465.png')

In [13]:
def get_key_items(data):
    for key in data_dict.keys():
        to_find = data_dict[key]['label'].lower()
  

        for i in range(len(data['text'])):
            if to_find in data['text'][i].lower():

                x,y,w,h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]

                if data_dict[key]['zoning_type'] == "in_line":
                    zoning_area = (x-10,y-10,w+x+600,h+y+10) 

                elif data_dict[key]['zoning_type'] == "bottom_large":
                    zoning_area = (x-w,y-10,w+x+w,h+y+50) 
                else:
                    zoning_area = (x-150,y+20,w+x+430,h+y+100)

#                 cv2.rectangle(img,(zoning_area[0],zoning_area[1]),(zoning_area[2],zoning_area[3]),(255,0,0),3) 

                temp_string = ""
                if data_dict[key]['value'] == "":
                    for j in range(len(data['text'])):
                        x,y,w,h = data['left'][j], data['top'][j], data['width'][j], data['height'][j]
                        if x>=zoning_area[0] and y>=zoning_area[1] and x+w <= zoning_area[2] and y+h <= zoning_area[3]:

                            temp_string += data['text'][j]+' '



                    if re.findall(data_dict[key]['Pattern'], temp_string):

                        data_dict[key]['value'] = re.findall(data_dict[key]['Pattern'], temp_string)[0]
                        final_op = {key:data_dict[key]['value'] for key in data_dict.keys()}
                   
                    break
    return final_op  
    
result=get_key_items(extracted_data)

In [14]:
result

{'Customer': '10012844',
 'Invoice': '',
 'Date': '08/10/2021',
 'Accounting CLERK NUMBER': '',
 'products': '301.00'}